In [1]:
import csv
from collections import Counter

data = []
keys = []
text = []
with open('canadian_blockade_articles.csv', 'r') as csvfile:
    data = [row for row in csv.DictReader(csvfile)]

keys = [key for key in data[0]]
print("Key for this data are: ", keys)

# Extract data
sentences = [item['body'] for item in data]
titles = [item['title'] for item in data]
sources = [item['source.title'] for item in data]

# Calculate statistics
src_counts = Counter(sources)
unique_titles = Counter(titles)

# Display statistics
print("There are {} sources".format(len(src_counts)))
print("There are {} articles, {} of which are unique".format(len(titles), len(unique_titles)))


Key for this data are:  ['ï»¿uri', 'url', 'date', 'time', 'source.uri', 'source.title', 'lang', 'title', 'body']
There are 5 sources
There are 100 articles, 100 of which are unique


In [4]:
import nltk
import gensim

from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

def sent_processor(sent, stopwords = {}):
    return [word for word in gensim.utils.simple_preprocess(sent, deacc=True) if word not in stopwords]

tokenized_titles = [sent_processor(title,stopwords) for title in titles]

In [20]:
KEY_WORD_THRESHOLD = 2

# This is a bit hardcore, it removes EVERY duplicate item
# TODO: Keep the latest article if a duplicate is found
def duplicateRemoval(data):
    return [item for i, item in enumerate(data) if item not in data[:i] + data[i+1:]]

def occurenceCount(title, filter_words):
    return sum([filter_word in title for filter_word in filter_words])

def flatten_list(lists):
    return [item for nested_list in lists for item in nested_list]

# TODO: Automatically create this word list
key_words = set(['railway','rail','railroad','canada','trudeau','mohawk','protest','indigenous','pipeline','blockade'])

# Filter articles
scored_titles = [(data, occurenceCount(title,key_words)) for data, title in zip(data, tokenized_titles)]
filtered_articles = [data for data,key_word_occ in scored_titles if KEY_WORD_THRESHOLD <= key_word_occ]
filtered_len = len(filtered_articles)
filtered_articles = duplicateRemoval(filtered_articles)

# Report filtering results
deduplicated_len = len(filtered_articles)
articles_filtered = len(data) - filtered_len
duplicate_articles = filtered_len - deduplicated_len
print("There were {} articles filtered and {} duplicates found".format(articles_filtered, duplicate_articles))
#duplicateRemoval(["the","that","this","the"])

There were 86 articles filtered and 0 duplicates found


['that', 'this']

In [27]:
import os
ROOT_PATH = "data"
article_ids = []

for article in filtered_articles:
    article_id = article['ï»¿uri']
    article_ids.append(article_id)
    with open(os.path.join(ROOT_PATH, article_id + '.txt'),'w+',encoding='utf-8') as article_fh:
        article_fh.writelines(article['body'])

with open(os.path.join(ROOT_PATH,'filelist.txt'),'w+') as lst_fh:
    lst_fh.writelines(['{}\n'.format(article) for article in article_ids])

In [4]:
# Utils to sort


# Create vocab
from gensim.corpora import Dictionary
dct = Dictionary(processed_text)
corpus = [dct.doc2bow(line) for line in processed_text]

print("There are {} unique words".format(len(dct)))

There are 1900 unique words


In [5]:
from gensim.models import TfidfModel
from gensim.matutils import corpus2dense
#TODO: Investigate using a sparse matrix (From a theoritical point of view)
model = TfidfModel(corpus)
tfidfs = [model[item] for item in corpus]
tfidfs = corpus2dense(tfidfs,len(dct),len(tfidfs))

In [6]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10)
kmeans.fit_predict(tfidfs.T)
Counter(kmeans.predict(tfidfs.T))

Counter({6: 14, 2: 12, 9: 13, 3: 14, 0: 4, 1: 9, 8: 11, 5: 3, 4: 12, 7: 8})